In [ ]:
import ipywidgets as wdg
from aitw_aiida_viscosity.cli.workflows.viscosity import launch_workflow

from common import VISCOSITY_WORKCHAIN_DEFAULTS, VISCOSITY_CODES, VISCOSITY_CODE_MAP

import utilities
import utilities_aiida as ua

In [ ]:
create_codes_button = wdg.Button(description='Create Codes')
output_area = wdg.Output()
def on_create_codes_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Ensure required computer/codes are set up...")
        
        profile = ua.setup_profile()
        computer = ua.setup_computer()
        codes = ua.setup_codes(code_infos=VISCOSITY_CODES)

        print("Using profile:")
        print(f" - {profile}")
        print("Using computer:")
        print(f" - {computer}")
        print("Using codes: ")
        for code in codes:
            print(f" - {code}")

create_codes_button.on_click(on_create_codes_button_clicked)
display(create_codes_button, output_area)

In [3]:
widgets = utilities.get_widgets_from_click_function(
    launch_workflow,
    override_defaults=VISCOSITY_WORKCHAIN_DEFAULTS,
    code_map=VISCOSITY_CODE_MAP,
)

title = wdg.HTML(value="<h2>AITW - AiiDA Monomer Viscosity Workchain</h2>")
display(title)
widget_box = wdg.VBox(list(widgets.values()))
display(widget_box)

generate_button = wdg.Button(description="Launch Workchain", button_style="success")
output_area = wdg.Output()

def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        args = utilities.get_click_args_from_widgets(launch_workflow, widgets)
        launch_workflow(args, standalone_mode=False)

generate_button.on_click(on_generate_button_clicked)
display(generate_button)
display(output_area)

HTML(value='<h2>AITW - AiiDA Monomer Viscosity Workchain</h2>')

Button(button_style='success', description='Launch Workchain', style=ButtonStyle())

Output()